Start with:
Data Check
Data cleansing
Verification that everything is proper

Analysis to do:
Country Analysis - Which country uses the most. BY $ amount and quantity
Product  Analytics  - Top products per country  
Price Analytics - By Month/Country
What sells the most, What is returned the most
Monthly ratio of successful sold to cancelled
Distribution between cancelled and successful invoice
Analysis of invoice by hourly distribution
Who are our top customers? By unit price, by quantity etc. Understand their needs and if data is skewed because of them eg. Big companies making big orders on a given product. (Might not mean it is favored by everyone). Basically check uniqueness in customer for given product.

In [48]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import random
import os
import glob

%matplotlib inline
# Make plots larger
plt.rcParams['figure.figsize'] = (15, 9)

In [49]:
ds = pd.read_excel("data_science_analytics_2018_data.xlsx")
#Save Orignal Data

In [50]:
ds.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,C581569,20979,36 PENCILS TUBE RED RETROSPOT,-5,2011-12-09 11:58:00,1.25,17315.0,United Kingdom
1,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom
2,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
3,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
4,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,-12,2011-12-09 09:57:00,1.95,14397.0,United Kingdom


In [51]:
cpy =ds

In [52]:
ds.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [53]:
ds.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [54]:
ds.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

ds.dropna(axis =0, subset=['CustomerID'],inplace=True)

In [55]:
ds.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [73]:
ds_top=ds[ds['InvoiceNo'].str.contains("C|c")==False]
ds_top.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,StockCode_total
9288,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.06,NaN,United Kingdom,-11062.06
9289,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.06,NaN,United Kingdom,-11062.06
9290,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom,11062.06


In [57]:
ds['InvoiceNo'].dtypes

dtype('O')

In [58]:
ds['StockCode'].value_counts()

85123A      2313
22423       2203
85099B      2159
47566       1727
20725       1639
84879       1502
22720       1477
22197       1476
21212       1385
20727       1350
22383       1348
22457       1280
23203       1267
POST        1256
22386       1251
22469       1239
22960       1229
21931       1214
22086       1210
22411       1202
20728       1197
22382       1192
22961       1182
22666       1180
23298       1179
22699       1138
22384       1137
23209       1135
82482       1129
22993       1111
            ... 
85023C         1
21923          1
84875A         1
46037A         1
85036a         1
21858          1
21854          1
84964B         1
84966A         1
72038P         1
84876C         1
37477B         1
85160a         1
22275          1
78027          1
DCGS0074       1
44242A         1
79323GR        1
84661b         1
51014c         1
22143          1
72800F         1
37477D         1
DCGS0073       1
79320          1
79341          1
84031a         1
10123G        

In [59]:
ds['StockCode_total'] = ds['Quantity'] * ds['UnitPrice']  

In [60]:
ds.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,StockCode_total
0,C581569,20979,36 PENCILS TUBE RED RETROSPOT,-5,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,-6.25
1,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom,-1.25
2,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom,-54.75
3,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom,-224.69
4,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,-12,2011-12-09 09:57:00,1.95,14397.0,United Kingdom,-23.40


In [61]:
ds.groupby(['StockCode'])[["StockCode_total"]].sum()

,StockCode_total
StockCode,
10002,759.890
10080,119.090
10120,40.530
10125,994.840
10133,1540.020
10134,0.000
10135,2206.140
11001,2152.390
15030,41.470


In [62]:
ref = ds.sort_values(['StockCode_total'],ascending=[0])

In [63]:
ref

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,StockCode_total
10771,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,168469.60
481328,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,77183.60
323242,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10 15:28:00,649.50,15098.0,United Kingdom,38970.00
527069,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,13541.33,NaN,United Kingdom,13541.33
9290,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,NaN,United Kingdom,11062.06
371728,551697,POST,POSTAGE,1,2011-05-03 13:46:00,8142.75,16029.0,United Kingdom,8142.75
199841,567423,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,1412,2011-09-20 11:05:00,5.06,17450.0,United Kingdom,7144.72
490156,540815,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,2011-01-11 12:55:00,2.10,15749.0,United Kingdom,6539.40
384332,550461,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,2011-04-18 13:20:00,2.10,15749.0,United Kingdom,6539.40
128077,573003,23084,RABBIT NIGHT LIGHT,2400,2011-10-27 12:11:00,2.08,14646.0,Netherlands,4992.00
